### Determine whether Scrambled Trees are Equivalent

Given two trees which represent a scrambled word. Determine if they're equivalent.

Given a string s1, we may represent it as a binary tree by partitioning it to two non-empty substrings recursively.

For example, s1 = "great":

To scramble the string, we may choose any non-leaf node and swap its two children.
For example, if we choose the node "gr" and swap its two children, it produces a scrambled string "rgeat".

We say that "rgeat" is a scrambled string of "great".
Similarly, if we continue to swap the children of nodes "eat" and "at", it produces a scrambled string "rgtae".

We say that "rgtae" is a scrambled string of "great".
Given two strings s1 and s2 of the same length, determine if s2 is a scrambled string of s1.

See [The example problem](https://leetcode.com/problems/scramble-string/description/)

In [200]:
debugging = False
#debugging = True

logging = True

def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))

def dbg_cont(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args), end='')

def log(f, *args):
    if logging:
        print((f).format(*args))
        
def logError(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))
        
def className(instance):
    return type(instance).__name__

### The TestSet Mechanism

In [201]:
# %load TestHarness.py
class TestCase(object):
    def __init__(self, name, method, inputs, expected, catchExceptions=False):
        self.name = name
        self.method = method
        self.inputs = inputs
        self.expected = expected
        self.catchExceptions = catchExceptions
        
    def run(self):
        if self.catchExceptions:
            try:
                return self.method(*self.inputs)
            except Exception as x:
                return x
        else:
                return self.method(*self.inputs)

import time
from datetime import timedelta

class TestSet(object):
    def __init__(self, cases):
        self.cases = cases
    
    def run_tests(self):
        count = 0
        errors = 0
        total_time = 0
        for case in self.cases:
            count += 1
            start_time = time.time()
            result = case.run()
            elapsed_time = time.time() - start_time
            total_time += elapsed_time
            if callable(case.expected):
                if not case.expected(result):
                    errors += 1
                    logError("Test {0} failed. Returned {1}", case.name, result)
            elif result != case.expected:
                errors += 1
                logError('Test {0} failed. Returned "{1}", expected "{2}"', case.name, result, case.expected)
        if errors:
            logError("Tests passed: {0}; Failures: {1}", count-errors, errors)
        else:
            log("All {0} tests passed.", count)
        log("Elapsed test time: {0}", timedelta(seconds=total_time))
        

In [202]:
14 % 2

0

In [203]:
14 // 2

7

### The Unit Under Test

In [204]:
import math

class AutoList(list):
    def __setitem__(self, index, value):
        if index >= len(self):
            self.extend([None]*(index + 1 - len(self)))
        list.__setitem__(self, index, value)
        
class ATree(object):
    """ Array representation of binary tree """
    val = AutoList()

    def __init__(self, v):
        self.text = v

    def sink(self, index=0):
        """ propagate tree changes down the tree """
        if index == 0:
            self.val[1] = self.text
            self.sink(index=1)
        else:
            assert index < 100
            v = self.val[index]
            dbg("{0} Sinking[{1}] == '{2}'", '-'*int(math.floor(math.log(index,2))), index, v)
            n = len(v)
            if n>1:
                offset = len(v) // 2
                vleft  = v[0:offset]
                vright = v[offset:]
                self.val[index*2 + 0] = vleft
                self.val[index*2 + 1] = vright
                if len(vleft) > 0: 
                    self.sink(index*2 + 0)
                if len(vright) > 0: 
                    self.sink(index*2 + 1)

    def swim(self, index=0):
        """ propagate tree changes up the tree """
        if index == 0:          
            self.text = self.val[1]
        else:
            assert index < 100            
            v = self.val[index]          
            vleft  = self.val[index*2 + 0]
            vright = self.val[index*2 + 1]
            newv = vleft + vright
            self.val[index] = newv
            dbg("{0} Swimming[{1}] == '{2}'==>{3}", '-'*int(math.floor(math.log(index,2))), index, v, newv)
            self.swim(index // 2)

    def exch(self, index=0):
        assert index > 0 # no exchanging at the text root
        assert index < len(self.val)
        v = self.val[index]       
        if v is None or len(v) < 2:
            return # ignore exchange of leaf nodes
        vleft  = self.val[index*2 + 0]
        vright = self.val[index*2 + 1]
        self.val[index*2 + 0] = vright
        self.val[index*2 + 1] = vleft
        self.swim(index)
        self.sink(index)

    def variants(self):
        pass
    

In [205]:
class SolutionBad(object):
    def isScramble(self, s1, s2):
        """
        :type s1: str
        :type s2: str
        :rtype: bool
        """
        
        if len(s1) != len(s2):
            return False

        if len(s1) == 0:
            return True

        if len(s1) == 1:
            return True if s1[0] == s2[0] else False

        t1 = ETree(s1).load()
        t2 = ETree(s2).load()
        print("T1={0}".format(t1.liststr()))
        print("T2={0}".format(t2.liststr()))
        return t1.isequiv(t1, t2)
        # This solution fails because the comparison tree doesn't have the same
        # structure as the source tree. No amount of flipping will make them equivalent.

In [222]:
class Solution(object):

    def isScramble(self, s1, s2):
        """
        :type s1: str
        :type s2: str
        :rtype: bool
        """
        if len(s1) != len(s2):
                return False
        
        if len(s1) == 0:
            return True
        
        if len(s1) == 1:
            return True if s1[0] == s2[0] else False
        
        # Ignore strings that don't contain the same set of characters.
        x = s2
        for c in s1:
            i = x.find(c)
            if i < 0:
                return False
            x = x[0:i] + x[i+1:]
            
        # Now try all tree operations until success or failure
        t = ETree(s1)
        v = t.variants()
        return True if s2 in t.variants() else False


In [223]:
t = ETree("great")
t = ETree("abcd")
t = ETree("abb")

print(t.liststr())
t.indexed(3).liststr()

#print('This is\na\ntest\nof the\nemergency broadcast\nsystem.')

"abb"==>[2:"a"], [3:"bb"==>[6:"b"], [7:"b"]]


'"bb"==>[6:"b"], [7:"b"]'

In [224]:
print(t.liststr())
x = t.variants()
print(x)
print(t.liststr())
"bab" in x

"abb"==>[2:"a"], [3:"bb"==>[6:"b"], [7:"b"]]
['bba', 'bba', 'bba', 'abb']
"abb"==>[2:"a"], [3:"bb"==>[6:"b"], [7:"b"]]


False

In [225]:
t.indexed(7).root().liststr()
t.indexed(7).liststr()
t.exch(7)
t.exch()
t.liststr()
print(t.listTree())

[]


### The Test Cases

In [187]:
t = ATree("great")
t.sink()
print(t.val)

t.exch(2)
t.exch(3)
#t.exch(6)

print(t.val)

  DBG: Sinking[1] == 'great'
  DBG:- Sinking[2] == 'gr'
  DBG:-- Sinking[4] == 'g'
  DBG:-- Sinking[5] == 'r'
  DBG:- Sinking[3] == 'eat'
  DBG:-- Sinking[6] == 'e'
  DBG:-- Sinking[7] == 'at'
  DBG:--- Sinking[14] == 'a'
  DBG:--- Sinking[15] == 't'
[None, 'great', 'gr', 'eat', 'g', 'r', 'e', 'at', None, None, None, None, None, None, 'a', 't']
  DBG:- Swimming[2] == 'gr'==>rg
  DBG: Swimming[1] == 'great'==>rgeat
  DBG:- Sinking[2] == 'rg'
  DBG:-- Sinking[4] == 'r'
  DBG:-- Sinking[5] == 'g'
  DBG:- Swimming[3] == 'eat'==>ate
  DBG: Swimming[1] == 'rgeat'==>rgate
  DBG:- Sinking[3] == 'ate'
  DBG:-- Sinking[6] == 'a'
  DBG:-- Sinking[7] == 'te'
  DBG:--- Sinking[14] == 't'
  DBG:--- Sinking[15] == 'e'
[None, 'rgate', 'rg', 'ate', 'r', 'g', 'a', 'te', None, None, None, None, None, None, 't', 'e']


In [43]:
Solution().isScramble("abb", "bab")

T1=abb==>(a), (bb==>(b), (b))
T2=bab==>(b), (ab==>(a), (b))
  DBG:-- isequiv('abb', 'bab')  DBG:-- Recurse
  DBG:-- isequiv('a', 'b')  DBG: Chars Same --> False
  DBG:-- isequiv('a', 'ab')  DBG:-- Recurse
  DBG: DIFF --> False
  DBG: DIFF --> False
  DBG: Answer --> False
  DBG: Answer --> False


False

In [44]:
Solution().isScramble("great", "rgtae")

T1=great==>(gr==>(g), (r)), (eat==>(e), (at==>(a), (t)))
T2=rgtae==>(rg==>(r), (g)), (tae==>(t), (ae==>(a), (e)))
  DBG:-- isequiv('great', 'rgtae')  DBG:-- Recurse
  DBG:-- isequiv('gr', 'rg')  DBG:-- Recurse
  DBG:-- isequiv('g', 'r')  DBG: Chars Same --> False
  DBG:-- isequiv('g', 'g')  DBG: Chars Same --> True
  DBG:-- isequiv('r', 'r')  DBG: Chars Same --> True
  DBG: Answer --> True
  DBG:-- isequiv('eat', 'tae')  DBG:-- Recurse
  DBG:-- isequiv('e', 't')  DBG: Chars Same --> False
  DBG:-- isequiv('e', 'ae')  DBG:-- Recurse
  DBG: DIFF --> False
  DBG: DIFF --> False
  DBG: Answer --> False
  DBG: Answer --> False
  DBG:-- isequiv('gr', 'tae')  DBG:-- Recurse
  DBG:-- isequiv('g', 't')  DBG: Chars Same --> False
  DBG:-- isequiv('g', 'ae')  DBG:-- Recurse
  DBG: DIFF --> False
  DBG: DIFF --> False
  DBG: Answer --> False
  DBG: Answer --> False
  DBG: Answer --> False


False

In [264]:
#simpletest = lambda A : [spiral_square(A)]
def simpletest(w1, w2):
    x = Solution()
    return x.isScramble(w1, w2)

c1g = TestCase('one same char', simpletest, ["a", "a"], True)
c1b = TestCase('one diff char', simpletest, ["a", "b"], False)

c2g = TestCase('two same char', simpletest, ["aa", "aa"], True)
c2f = TestCase('two flip char', simpletest, ["ab", "ba"], True)
c2b = TestCase('two diff char', simpletest, ["ab", "bc"], False)


c3g  = TestCase('three same char', simpletest, ["abc", "abc"], True)
c3f  = TestCase('three flip char', simpletest, ["abb", "bba"], True)
c3f2 = TestCase('three flip char', simpletest, ["abb", "bab"], True)

tester = TestSet([c1g, c1b, c2g, c2f, c2b, c3g, c3f, c3f2])


In [265]:
tester.run_tests()

*** ERROR:Test three flip char failed. Returned "False", expected "True"
*** ERROR:Tests passed: 7; Failures: 1
Elapsed test time: 0:00:00.000684


In [21]:
index=17
math.log(index,2)
math.floor(math.log(index,2))
4.0

4.0

In [ ]:
BoolArray3D(object):
    val = []
    
    def __init__(self, rows, cols, z):
        inner = [0]*z
    
    
    

In [267]:
def isScramble(s1, s2):
    """
        Let F(i, j, k) = whether the substring S1[i..i + k - 1] is a scramble of S2[j..j + k - 1] or not
        Since each of these substrings is a potential node in the tree, we need to check for all possible cuts.
        Let q be the length of a cut (hence, q < k), then we are in the following situation:
    
         S1 [   x1    |         x2         ]
            i         i + q                i + k - 1
 
         here we have two possibilities:

         S2 [   y1    |         y2         ]
            j         j + q                j + k - 1
   
                     or 
 
        S2 [       y1        |     y2     ]
           j                 j + k - q    j + k - 1
 
         which in terms of F means:
 
         F(i, j, k) = for some 1 <= q < k we have:
         (F(i, j, q) AND F(i + q, j + q, k - q)) OR (F(i, j + k - q, q) AND F(i + q, j, k - q))
  
         Base case is k = 1, where we simply need to check for S1[i] and S2[j] to be equal 
    """
    slen = len(s1) 
    if slen != len(s2): return False
    #boolean [][][] F = new boolean[len][len][len + 1];
    F = [[[False]*(slen+1)]*slen]*slen
    k = 1
    while k<=slen: #for (int k = 1; k <= len; ++k)
        i = 0
        while i+k <= slen:  #for (int i = 0; i + k <= len; ++i)
            j = 0
            while j+k <= slen: #for (int j = 0; j + k <= len; ++j)
                if k == 1:
                    F[i][j][k] = s1[i] == s2[j]
                else:
                    q = 1
                    while q<k and not F[i][j][k]: #for (int q = 1; q < k && !F[i][j][k]; ++q) {
                        t1 = F[i][j        ][q] and F[i + q][j + q][k - q]
                        t2 = F[i][j + k - q][q] and F[i + q][j    ][k - q]
                        F[i][j][k] = t1 or t2
                        q += 1
                j += 1
            i += 1
        k += 1
    print(F)
    return F[0][0][slen]


In [269]:
isScramble("ab", "ba")

[[[False, False, False], [False, False, False]], [[False, False, False], [False, False, False]]]


False

### Some Ad Hoc Tests

In [257]:
x = [[None]]*10 
print(x)
x[0].append('happy trails')
x[0] += 'WTF'
x += 'GLORM'
x
[x for x in range(1, 3)]

[[None], [None], [None], [None], [None], [None], [None], [None], [None], [None]]


[1, 2]

In [241]:
[[[0]*4]*3]*2

[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]